In [10]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

import os
import numpy as np
from PIL import Image
import glob

from sklearn.model_selection import train_test_split
#import splitfolders  # 데이터를 train, val, test폴더로 분리
from tqdm import trange, notebook

In [2]:
# 데이터 폴더 경로
path = 'C:\\Users\\JungHyeona\\Documents\\Project\\AI_deeplearning\\image'

In [3]:
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        img = self.images[index]
        label = self.labels[index]
        
        if self.transform is not None:
            img = self.transform(img)
            
        return img, label

In [4]:
images = []
labels = []

for for_name in os.listdir(path):  # i = ['Broken', 'Normal']
    forder = os.path.join(path, for_name)  # ex) path/Normal

    for j in os.listdir(forder):
        file = os.path.join(forder, j)  # ex) path/Normal/a.png
        
        image = Image.open(file)
        label = os.path.basename(os.path.dirname(file))
        
        images.append(image)
        labels.append(label)

In [5]:
# 라벨 값 변경
for i in range(len(labels)):
    if labels[i] == 'Normal':
        labels[i] = 0
    else:
        labels[i] = 1

In [6]:
len(labels)

2110

In [21]:
# 데이터 전처리
transform = transforms.Compose([
     transforms.Resize((32, 32)),
     transforms.ToTensor(),
     transforms.Normalize((0.111850675, 0.11192831, 0.11306531), (0.029490899, 0.0293483175, 0.029255329999999996))])

In [22]:
# 데이터셋 인스턴스 생성
dataset = CustomDataset(images, labels, transform=transform)

In [23]:
# 데이터셋을 train, validation, test로 분리
train_dataset, test_dataset = train_test_split(dataset, test_size=0.1, random_state=42)
train_dataset, val_dataset = train_test_split(train_dataset, test_size=0.1, random_state=42)

In [24]:
# DataLoader를 통해 데이터를 불러올 수 있도록 설정
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# 모델

In [13]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.model = nn.Sequential(
            # (32, 32, 3) -> (32, 32, 32), (16, 16, 32)
            nn.Conv2d(3, 64, kernel_size=(3, 3), padding=1, stride=1),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),
            
            # (16, 16, 32) -> (16, 16, 64) -> (8, 8, 64)
            nn.Conv2d(32, 64, kernel_size=(3, 3), padding=1, stride=1),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),
            
            # (8, 8, 64) -> (8, 8, 128) -> (4, 4, 128)
            nn.Conv2d(64, 128, kernel_size=(3, 3), padding=1, stride=1),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),
            
            nn.Flatten(),
            nn.Linear(4*4*128, 18),
            nn.Linear(18, 3)
        )
        

    def forward(self,x):
    	# self.layer에 정의한 연산 수행
        out = self.model(x)
        return out

In [25]:
# B모드
class CNN(nn.Module):
    
    def __init__(self):
        
        super(CNN, self).__init__()
        self.model = nn.Sequential(
            # (3, 32, 32) -> (32, 32, 32)
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),  #cnn layer
            nn.ReLU(),
            nn.BatchNorm2d(32),    #batch layer
            nn.MaxPool2d(kernel_size=2, stride=2),    #pooling layer  (32, 16, 16)

            # (32, 16, 16) -> (64, 16, 16)
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),  #cnn layer
            nn.ReLU(),
            nn.BatchNorm2d(64),    #batch layer
            nn.MaxPool2d(kernel_size=2),    #pooling layer  (64, 8, 8)
            
            # (64, 8, 8) -> (128, 8, 8)
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),  #cnn layer
            nn.ReLU(),
            nn.BatchNorm2d(128),    #batch layer
            nn.MaxPool2d(kernel_size=2),    #pooling layer  (128, 4, 4)
            nn.Dropout2d(p=0.5),
            
            nn.Flatten(),
            nn.Linear(128*4*4, 100),
            nn.ReLU(),
            nn.Linear(100, 2)
        )


    def forward(self, x):
        out = self.model(x)l
        return out

In [34]:
model = CNN()

# GPU 사용 가능 여부에 따라 디바이스 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

model.to(device)

In [15]:
loss_func = nn.CrossEntropyLoss()
learning_rate = 0.0002
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

In [27]:
total_batch = len(train_loader)
print('총 배치의 수 : {}'.format(total_batch))

num_epochs = 10
for epoch in notebook.tqdm(range(num_epochs)):
    avg_cost = 0

    for X, Y in train_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X#.to(device)
        Y = Y#.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = loss_func(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

총 배치의 수 : 54


  0%|          | 0/10 [00:00<?, ?it/s]

[Epoch:    1] cost = 0.650311768
[Epoch:    2] cost = 0.650252938
[Epoch:    3] cost = 0.656199515
[Epoch:    4] cost = 0.643773794
[Epoch:    5] cost = 0.640102267
[Epoch:    6] cost = 0.644215703
[Epoch:    7] cost = 0.66146642
[Epoch:    8] cost = 0.647153735
[Epoch:    9] cost = 0.653293014
[Epoch:   10] cost = 0.651719987


In [37]:
# 모델 훈련
num_epochs = 3

for epoch in notebook.tqdm(range(num_epochs)):
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    
    for images, labels in train_loader:
        images = images#.to(device)
        labels = labels#.to(device)
        
        # 순전파
        outputs = model(images)
        loss = loss_func(outputs, labels)
        
        # 역전파 및 최적화
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # 정확도 계산
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)
        
        # 통계 출력
        running_loss += loss.item() * images.size(0)
    
    epoch_loss = running_loss / len(train_dataset)
    epoch_accuracy = correct_predictions / total_predictions * 100
    
    print(f'Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f} - Accuracy: {epoch_accuracy:.2f}%')

  0%|          | 0/3 [00:00<?, ?it/s]

NameError: name 'trainset' is not defined

In [32]:
# 모델 평가 함수
# def evaluate(model, dataloader, device):
def evaluate(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            #inputs, labels = inputs.to(device), labels.to(device)
            inputs, labels = inputs, labels
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

In [29]:
# 모델 평가 : cuda로 cpu사용 시
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
val_accuracy = evaluate(model, val_loader, device)
print(f'Validation Accuracy: {val_accuracy:.2f}%')

Validation Accuracy: 68.95%


In [35]:
# 모델 평가
val_accuracy = evaluate(model, val_loader)
print(f'Validation Accuracy: {val_accuracy:.2f}%')

Validation Accuracy: 71.05%


In [19]:
# 모델 저장
torch.save(model, path+'cnn.pt')

# test